In [1]:
import os 
import sys

In [2]:
import random
random.seed(999)
random.uniform(0, 1)

0.7813468849570298

In [3]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import librosa, librosa.display
import resampy

import soundfile as sf 
import pickle 

from vad import *

In [4]:
from spectrogram import generate_log_spectrogram

In [5]:
audiofile = 'giulbia-data/301 - Crying baby/1-187207-A.ogg'
wav_data, sr = sf.read(audiofile)
print(wav_data[:10])
print(sr)
wav_data = resampy.resample(wav_data, sr, 16000)
print(wav_data[:10])

'''
wav_data1, sr1 = sf.read(audiofile,dtype='int16')
print(wav_data1[:10])
print(sr1)
wav_data1 = wav_data1/32768 
wav_data1 = resampy.resample(wav_data1, sr1, 16000)
print(wav_data1[:10])
'''

wav_data2, sr2 = librosa.load(audiofile,sr=16000)
print(wav_data2[:10])
print(sr2)


[0.00012519 0.00012013 0.00012783 0.00015009 0.0001558  0.00015325
 0.00015205 0.00015739 0.00018016 0.00018845]
44100
[8.07522269e-05 1.52612243e-04 1.49930745e-04 1.79956849e-04
 1.85337242e-04 1.84350326e-04 1.84456538e-04 1.81405155e-04
 1.54161099e-04 1.54188430e-04]
[8.0752230e-05 1.5261221e-04 1.4993075e-04 1.7995681e-04 1.8533722e-04
 1.8435031e-04 1.8445663e-04 1.8140518e-04 1.5416119e-04 1.5418846e-04]
16000


In [6]:
generate_log_spectrogram(audiofile,'./',sr=8000,segment=4.1, pre_emphasis=None,
                                    n_mels=64,fmin=0,fmax=None,                                     
                                    n_fft=512, hop_length=512,debug=True,VAD=True)

giulbia-data/301 - Crying baby/1-187207-A.ogg 1-187207-A.ogg
5.0 4.1 5.0 1
0 0.0 True (64, 64)


[(array([[ 33.837833,  77.11378 ,  75.271324, ...,  95.2613  ,  58.856934,
           28.95661 ],
         [ 35.191074,  70.63287 ,  54.268505, ..., 102.95239 ,  60.61605 ,
           60.874825],
         [ 32.674988,  53.652634,  85.75292 , ...,  74.3843  ,  60.221043,
           54.937027],
         ...,
         [  0.      ,  42.531986,  22.996464, ...,   0.      ,   0.      ,
            0.      ],
         [  0.      ,  28.943624,   8.315365, ...,   0.      ,   0.      ,
            0.      ],
         [  0.      ,   0.      ,   0.      , ...,   0.      ,   0.      ,
            0.      ]], dtype=float32),
  0,
  True)]

In [7]:

train_cry_sources = ['donateacry-corpus/deepcam-cleaned/crys-5s-segments/',
               'AudioSet/Babycry-infantcry-cleaned/babycry-5s-segments/'
              ] 

train_nocry_sources = [
                 'ESC-50/audio/',
                 'donateacry-corpus/deepcam-cleaned/filtered-out/',
                 'AudioSet/others_train_5s_segments/'
                ] 

valid_cry_sources = ['giulbia-data/301 - Crying baby/']
valid_nocry_sources = ['giulbia-data/901 - Silence/',
                       'giulbia-data/902 - Noise/',
                       'giulbia-data/903 - Baby laugh/',
                       'AudioSet/others_valid_5s_segments/'
                      ]
                     
test_cry_sources = ['ESC-50/babycry/',
                   'AudioSet/babycry_eval_5s_segments/'
                   ]

test_nocry_sources = ['AudioSet/others_eval_5s_segments/']


In [8]:
'''
train_cry_sources = []

train_nocry_sources = [
                ] 

valid_cry_sources = ['giulbia-data/301 - Crying baby/']
valid_nocry_sources = ['giulbia-data/901 - Silence/',
                       'giulbia-data/902 - Noise/',
                       'giulbia-data/903 - Baby laugh/'
                       #'AudioSet/others_valid_5s_segments/'
                      ]
                     
test_cry_sources = ['ESC-50/babycry/']

test_nocry_sources = []
'''

"\ntrain_cry_sources = []\n\ntrain_nocry_sources = [\n                ] \n\nvalid_cry_sources = ['giulbia-data/301 - Crying baby/']\nvalid_nocry_sources = ['giulbia-data/901 - Silence/',\n                       'giulbia-data/902 - Noise/',\n                       'giulbia-data/903 - Baby laugh/'\n                       #'AudioSet/others_valid_5s_segments/'\n                      ]\n                     \ntest_cry_sources = ['ESC-50/babycry/']\n\ntest_nocry_sources = []\n"

In [9]:
dataset = 'dataset' 

train_cry_folder = dataset + '/train/cry/'  
train_nocry_folder = dataset + '/train/nocry/'  

valid_cry_folder = dataset + '/valid/cry/'  
valid_nocry_folder = dataset + '/valid/nocry/'  

test_cry_folder = dataset + '/test/cry/'  
test_nocry_folder = dataset + '/test/nocry/'  

if not os.path.exists(train_cry_folder):
    os.makedirs(train_cry_folder)
if not os.path.exists(train_nocry_folder):
    os.makedirs(train_nocry_folder)
if not os.path.exists(valid_cry_folder):
    os.makedirs(valid_cry_folder)
if not os.path.exists(valid_nocry_folder):
    os.makedirs(valid_nocry_folder)
if not os.path.exists(test_cry_folder):
    os.makedirs(test_cry_folder)
if not os.path.exists(test_nocry_folder):
    os.makedirs(test_nocry_folder)

    

In [10]:
cry_sources = [] 
cry_sources.append(train_cry_sources)
cry_sources.append(valid_cry_sources)
cry_sources.append(test_cry_sources)

noncry_sources = [] 
noncry_sources.append(train_nocry_sources)
noncry_sources.append(valid_nocry_sources)
noncry_sources.append(test_nocry_sources)


dataset = {"train": {"cry":[], "nocry":[] },
           "valid": {"cry":[], "nocry":[] },
           "test": {"cry":[], "nocry":[] }           
          }


In [11]:
sr=8000; segment=4.1; pre_emphasis=None
n_mels=64; fmin=0; fmax=None
n_fft=512; hop_length=512
VAD_on = True 


In [ ]:
for i in range(len(cry_sources)):
    source = cry_sources[i]    
    for s in source:
                
        files = os.listdir(s) 
        for f in files:
            
            r = random.uniform(0, 1)
            
            if r<0.8:
                dest = train_cry_folder
                dest_d = dataset["train"]["cry"]
            elif r<0.9:
                dest = valid_cry_folder
                dest_d = dataset["valid"]["cry"]
            else:
                dest = test_cry_folder
                dest_d = dataset["test"]["cry"]
                
            #print(s+f,dest)               
            melgrams = generate_log_spectrogram(s+f,dest,sr=sr,segment=segment,pre_emphasis=pre_emphasis,
                                    n_mels=n_mels,fmin=fmin,fmax=fmax,                                     
                                    n_fft=n_fft, hop_length=hop_length,VAD=VAD_on,debug=False)
            if len(melgrams)>0:
                dest_d.append((s+f,melgrams))


Reading audio file donateacry-corpus/deepcam-cleaned/crys-5s-segments/crys-5s-segments.zip has error
Reading audio file donateacry-corpus/deepcam-cleaned/crys-5s-segments/40a4c760-facf-4911-b0a6-22eccc3ab18d-1430477691332-1.7-f-26-hu.3gp_176_0.wav has error


In [14]:
for i in range(len(noncry_sources)):
    source = noncry_sources[i]    
    for s in source:
                
        files = os.listdir(s) 
        for f in files:
            
            r = random.uniform(0, 1)
            
            if r<0.8:
                dest = train_nocry_folder
                dest_d = dataset["train"]["nocry"]
            elif r<0.9:
                dest = valid_nocry_folder
                dest_d = dataset["valid"]["nocry"]
            else:
                dest = test_nocry_folder
                dest_d = dataset["test"]["nocry"]
                
            #print(s+f,dest)               
            melgrams = generate_log_spectrogram(s+f,dest,sr=sr,segment=segment,pre_emphasis=pre_emphasis,
                                    n_mels=n_mels,fmin=fmin,fmax=fmax,                                     
                                    n_fft=n_fft, hop_length=hop_length,VAD=VAD_on,debug=False)
            if len(melgrams)>0:
                dest_d.append((s+f,melgrams))

Reading audio file donateacry-corpus/deepcam-cleaned/filtered-out/F15D825B-F861-410B-ACC9-9863C8949217-1427012060-1.0-m-04-ti.caf has error
Reading audio file donateacry-corpus/deepcam-cleaned/filtered-out/F15D825B-F861-410B-ACC9-9863C8949217-1427012073-1.0-m-04-ti.caf has error
Reading audio file donateacry-corpus/deepcam-cleaned/filtered-out/01a592eb-513d-4431-b7ba-b0fba5a00908-1430339263195-1.7-m-04-hu.3gp has error
Reading audio file donateacry-corpus/deepcam-cleaned/filtered-out/F15D825B-F861-410B-ACC9-9863C8949217-1427012078-1.0-m-04-ti.caf has error
Reading audio file donateacry-corpus/deepcam-cleaned/filtered-out/c8fa621d-f18e-485d-9b23-c44fadb6e1d3-1430724173566-1.7-m-04-hu.3gp has error
Reading audio file donateacry-corpus/deepcam-cleaned/filtered-out/F15D825B-F861-410B-ACC9-9863C8949217-1427012192-1.0-m-04-ti.caf has error
Reading audio file donateacry-corpus/deepcam-cleaned/filtered-out/F15D825B-F861-410B-ACC9-9863C8949217-1427012070-1.0-m-04-ti.caf has error
Reading audio 

In [22]:
dataset_file = 'dataset/trainset_{}_{}_{}_{}_{}_{}_{}_{}.pk'.format(
    sr,segment,pre_emphasis,n_mels,fmin,fmax,n_fft,hop_length) 

print(dataset_file)
with open(dataset_file , 'wb') as pk_file:
                pickle.dump(dataset, pk_file)

dataset/trainset_8000_4.1_None_64_0_None_512_512.pk


In [16]:
test_dir = 'Self/10Cry/'
audiofiles = os.listdir(test_dir) 
audiofiles = [x for x in audiofiles if x.endswith('mp3')]
print(audiofiles)

['7376bd92-3ccd-4672-bf08-2bb5dd992b30.mp3', 'E062904F3842_subjectawake_1637580472817.mp3', 'E062904F3842_subjectawake_1637717631593.mp3', 'E062906A5A6D_subjectawake_1638712785921.mp3', '146B9CB78294_subjectawake_1620198310758.mp3', 'E06290646B49_subjectAsleep_1638305984190.mp3', 'E06290637154_breathingStopped_1637525474363.mp3', 'E06290634162_subjectawake_1638677765467.mp3', '146B9CB78A18_subjectawake_1620132682993.mp3', '146B9C6FA25A_monitoringOff_1619739616502.mp3']


In [17]:
#audiofiles = ['E06290646B49_subjectAsleep_1638305984190.mp3']

In [18]:
test_data = {"test":{"cry":[],"noncry":[]}
            }

for f in audiofiles:
    f_full = test_dir+f     
    print(f_full)
    melgrams = generate_log_spectrogram(f_full,None,sr=sr,duration=None,segment=segment,pre_emphasis=pre_emphasis,
                                    n_mels=n_mels,fmin=fmin,fmax=fmax,                                     
                                    n_fft=n_fft, hop_length=hop_length)
    
    if len(melgrams)>0:
        print(len(melgrams),melgrams[0][0].shape)
        test_data["test"]["cry"].append((f_full,melgrams))


Self/10Cry/7376bd92-3ccd-4672-bf08-2bb5dd992b30.mp3
38 (64, 64)
Self/10Cry/E062904F3842_subjectawake_1637580472817.mp3
60 (64, 64)
Self/10Cry/E062904F3842_subjectawake_1637717631593.mp3
61 (64, 64)
Self/10Cry/E062906A5A6D_subjectawake_1638712785921.mp3
62 (64, 64)
Self/10Cry/146B9CB78294_subjectawake_1620198310758.mp3
48 (64, 64)
Self/10Cry/E06290646B49_subjectAsleep_1638305984190.mp3
66 (64, 64)
Self/10Cry/E06290637154_breathingStopped_1637525474363.mp3
63 (64, 64)
Self/10Cry/E06290634162_subjectawake_1638677765467.mp3
62 (64, 64)
Self/10Cry/146B9CB78A18_subjectawake_1620132682993.mp3
59 (64, 64)
Self/10Cry/146B9C6FA25A_monitoringOff_1619739616502.mp3
34 (64, 64)


In [19]:
dataset_file = 'dataset/testset_{}_{}_{}_{}_{}_{}_{}_{}.pk'.format(
    sr,segment,pre_emphasis,n_mels,fmin,fmax,n_fft,hop_length) 

print(dataset_file)
with open(dataset_file , 'wb') as pk_file:
    pickle.dump(test_data, pk_file)        

dataset/testset_8000_4.1_None_64_0_None_512_512.pk


In [20]:
'''
dataset_file = 'dataset/testset_8000_5_None_64_0_None_256_128.pk'
with open(dataset_file, 'rb') as pk_file:
    data = pickle.load(pk_file)
    
for d in data:
    test_data["test"]["cry"].append((d[0],d[1]))
'''    

'\ndataset_file = \'dataset/testset_8000_5_None_64_0_None_256_128.pk\'\nwith open(dataset_file, \'rb\') as pk_file:\n    data = pickle.load(pk_file)\n    \nfor d in data:\n    test_data["test"]["cry"].append((d[0],d[1]))\n'